In [1]:
# Import dependencies

import numpy as np 
from citipy import citipy
import gmaps
import requests
import pandas as pd 
from datetime import datetime

from config import weather_api_key
from config import g_key

In [2]:
# Generate 2,000 random latitudes and longitudes, and zip them as a list of tuples
lats = np.random.uniform(-90.0,90.0, size=1500)
lngs = np.random.uniform(-180.0,180.0,size=1500)
lats_lngs = zip(lats, lngs)
coordinates = list(lats_lngs)

In [3]:
# Get the nearest city using citipy module
cities = list()
# Identify the nearest city for each latitude and longitude combination
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0],coordinate[1]).city_name
    
    # If the city is unique, then add to the cities list
    if city not in cities:
        cities.append(city)
# Print the city count to confirm sufficient control
print(cities[:10], 'Generate', len(cities))

['luderitz', 'hermanus', 'rokytne', 'khatanga', 'albany', 'rikitea', 'grand river south east', 'channel-port aux basques', 'barrow', 'havre'] Generate 589


In [4]:
# Perform an API call with the OpenWeatherMap.

# initial counters for log and sets
record_count = 1 
set_count = 1

city_data = list()
basic_url = "http://api.openweathermap.org/data/2.5/weather?units=imperial&APPID=" + weather_api_key

print('Beginning Data Retrieval     ')
print("-----------------------------")
# use enumerate() method to loop index and item simutanously 
for i, item in enumerate(cities):
    if i % 50 == 0 and i != 0 :
        record_count = 1  
        set_count +=1 
    # build URL for API call
    url = basic_url + '&q=' + item
    # Log the URL, record, and set numbers and the city.
    print(f'Processing Record {record_count} of Set {set_count} | {item}')
    # increment record count
    record_count +=1

    try:
        js = requests.get(url).json()
        city_name = item
        city_country = js['sys']['country']
        city_UTCdate = js['dt']
        city_date = datetime.utcfromtimestamp(city_UTCdate).strftime('%Y-%m-%d %H:%M:%S')
        city_lat = js['coord']['lat']
        city_lon = js['coord']['lon']
        city_max_temp = js['main']['temp_max']
        city_humidity = js['main']['humidity']
        city_cload = js['clouds']['all']
        city_wind_speed = js['wind']['speed']
        city_weather_description = js['weather'][0]['description']

        # raining info
        try:
            js['weather'][0]['main'] == 'Rain'
            city_rain = js['rain']['3h']
        except:
            city_rain = 0.000

        # snowing info
        try:
            js['weather'][0]['main'] == 'Snow'
            city_snow = js['snow']['3h']
        except:
            city_snow = 0    

        # append as a list of dictionaries
        city_data.append({'City':city_name, 'Country':city_country, 'Date':city_date, 
                        'Lat':city_lat, 'Lng': city_lon,'Max Temp':city_max_temp,
                        'Humidity':city_humidity,'Cloudiness': city_cload,'Wind Speed':city_wind_speed,
                        'Current Description':city_weather_description,
                        'Rain inches (last 3 hrs)':city_rain,'Snow inches (last 3 hrs)':city_snow})
    except:
        print("City not found. Skipping...")
        pass
print('-------------------------------')
print('Data Retrieval Complete        ')
print('-------------------------------')

print(len(city_data))

Beginning Data Retrieval     
-----------------------------
Processing Record 1 of Set 1 | luderitz
Processing Record 2 of Set 1 | hermanus
Processing Record 3 of Set 1 | rokytne
Processing Record 4 of Set 1 | khatanga
Processing Record 5 of Set 1 | albany
Processing Record 6 of Set 1 | rikitea
Processing Record 7 of Set 1 | grand river south east
City not found. Skipping...
Processing Record 8 of Set 1 | channel-port aux basques
Processing Record 9 of Set 1 | barrow
Processing Record 10 of Set 1 | havre
Processing Record 11 of Set 1 | vaini
Processing Record 12 of Set 1 | narsaq
Processing Record 13 of Set 1 | mar del plata
Processing Record 14 of Set 1 | camargo
Processing Record 15 of Set 1 | suraabad
Processing Record 16 of Set 1 | ushuaia
Processing Record 17 of Set 1 | upernavik
Processing Record 18 of Set 1 | ponta do sol
Processing Record 19 of Set 1 | inongo
Processing Record 20 of Set 1 | yellowknife
Processing Record 21 of Set 1 | saint-pierre
Processing Record 22 of Set 1 |

Processing Record 37 of Set 4 | juneau
Processing Record 38 of Set 4 | mahadday weyne
City not found. Skipping...
Processing Record 39 of Set 4 | zaysan
Processing Record 40 of Set 4 | sandwick
Processing Record 41 of Set 4 | arraial do cabo
Processing Record 42 of Set 4 | asosa
Processing Record 43 of Set 4 | cabo san lucas
Processing Record 44 of Set 4 | bargal
City not found. Skipping...
Processing Record 45 of Set 4 | yar-sale
Processing Record 46 of Set 4 | tateyama
Processing Record 47 of Set 4 | corinto
Processing Record 48 of Set 4 | linhares
Processing Record 49 of Set 4 | kotido
Processing Record 50 of Set 4 | durg
Processing Record 1 of Set 5 | provideniya
Processing Record 2 of Set 5 | porbandar
Processing Record 3 of Set 5 | srednekolymsk
Processing Record 4 of Set 5 | illoqqortoormiut
City not found. Skipping...
Processing Record 5 of Set 5 | rio grande
Processing Record 6 of Set 5 | atambua
Processing Record 7 of Set 5 | santa cruz cabralia
Processing Record 8 of Set 5 |

Processing Record 27 of Set 8 | hay river
Processing Record 28 of Set 8 | shitanjing
Processing Record 29 of Set 8 | mount gambier
Processing Record 30 of Set 8 | visby
Processing Record 31 of Set 8 | hualmay
Processing Record 32 of Set 8 | egvekinot
Processing Record 33 of Set 8 | monte alegre
Processing Record 34 of Set 8 | byron bay
Processing Record 35 of Set 8 | tautira
Processing Record 36 of Set 8 | aflu
City not found. Skipping...
Processing Record 37 of Set 8 | abonnema
Processing Record 38 of Set 8 | torbay
Processing Record 39 of Set 8 | luba
Processing Record 40 of Set 8 | santo domingo
Processing Record 41 of Set 8 | nouadhibou
Processing Record 42 of Set 8 | cockburn town
Processing Record 43 of Set 8 | banmankhi
Processing Record 44 of Set 8 | clarence town
Processing Record 45 of Set 8 | karratha
Processing Record 46 of Set 8 | evensk
Processing Record 47 of Set 8 | kawalu
Processing Record 48 of Set 8 | yangambi
Processing Record 49 of Set 8 | sorland
Processing Record

Processing Record 15 of Set 12 | san antonio
Processing Record 16 of Set 12 | marienburg
Processing Record 17 of Set 12 | puerto ayacucho
Processing Record 18 of Set 12 | abengourou
Processing Record 19 of Set 12 | larsnes
Processing Record 20 of Set 12 | faanui
Processing Record 21 of Set 12 | kozachi laheri
Processing Record 22 of Set 12 | bodden town
Processing Record 23 of Set 12 | lasa
Processing Record 24 of Set 12 | point fortin
Processing Record 25 of Set 12 | chik
Processing Record 26 of Set 12 | klyuchi
Processing Record 27 of Set 12 | abu samrah
Processing Record 28 of Set 12 | hokitika
Processing Record 29 of Set 12 | tessalit
Processing Record 30 of Set 12 | paamiut
Processing Record 31 of Set 12 | bara
Processing Record 32 of Set 12 | rohnert park
Processing Record 33 of Set 12 | morehead
Processing Record 34 of Set 12 | kakching
Processing Record 35 of Set 12 | te anau
Processing Record 36 of Set 12 | port lincoln
Processing Record 37 of Set 12 | kudahuvadhoo
Processing 

In [5]:
# Convert the array of dictionaries to a Pandas DataFrame.
WeatherPy_Database_df = pd.DataFrame(city_data)
new_column_order = ["City", "Country", "Lat", "Lng", "Max Temp", "Humidity", "Cloudiness", "Wind Speed", "Current Description"]
WeatherPy_Database_df = WeatherPy_Database_df[new_column_order]
WeatherPy_Database_df.head(10)

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,luderitz,NA,-26.6481,15.1594,65.93,78,0,18.37,clear sky
1,hermanus,ZA,-34.4187,19.2345,64.00,94,96,5.23,moderate rain
2,rokytne,UA,51.2796,27.2140,33.35,91,56,5.39,broken clouds
3,khatanga,RU,71.9667,102.5000,-20.45,95,100,10.92,overcast clouds
4,albany,US,42.6001,-73.9662,28.99,40,60,5.01,broken clouds
5,rikitea,PF,-23.1203,-134.9692,80.91,76,23,12.66,moderate rain
6,channel-port aux basques,CA,47.5702,-59.1367,20.25,82,100,18.12,overcast clouds
7,barrow,US,71.2906,-156.7887,-4.00,77,1,17.27,snow
8,havre,FR,49.5833,0.3333,46.40,93,90,18.41,overcast clouds
9,vaini,TO,-21.2000,-175.2000,84.20,74,75,9.22,broken clouds


In [6]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_Database.csv"
# 8b. Export the City_Data into a csv
WeatherPy_Database_df.to_csv(output_data_file, index_label="City_ID")